<a href="https://colab.research.google.com/github/van-dang/FEniCS-Colab/blob/master/StrongPeriodicBC_TwoComp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#About the solver

In [0]:
# This demo solves the transformed Bloch-Torrey equation applied to diffusion MRI 
# using the standard finite element method coupled with the theta-method 
# for the space discretization.

# This demo allows for a two-compartment domain with periodic boundaries and the 
# spins can enter and exit the computational domain freely by using the periodic 
# boundary conditions

# Copyright (C) 2017 Van-Dang Nguyen

# This file is part of DOLFIN.

# DOLFIN is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# DOLFIN is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
# GNU Lesser General Public License for more details.

# You should have received a copy of the GNU Lesser General Public License
# along with DOLFIN. If not, see <http://www.gnu.org/licenses/>.

# First added:  2017-10-10
# Last changed: 2019-04-10

# Setting a working environment

In [0]:
from google.colab import files

import platform, sys
python_version=platform.python_version()
from distutils.version import LooseVersion, StrictVersion

if ( LooseVersion(python_version) < LooseVersion("3.0.0")):
    print("Python3 is needed!");
    print("How to fix: Runtime/Change_runtime_type/Python 3");
    sys.exit()
    
try:
    from dolfin import *; from mshr import *
except ImportError as e:
    !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
    !add-apt-repository -y ppa:fenics-packages/fenics
    !apt-get update -qq
    !apt install -y --no-install-recommends fenics
    from dolfin import *; from mshr import *
    
import matplotlib.pyplot as plt;
from IPython.display import clear_output, display; import time; import dolfin.common.plotting as fenicsplot 
import time

import os, sys, shutil

dolfin_version = dolfin.__version__
print ('dolfin version:', dolfin_version)

!rm -rf * # clean up all files

# dolfin setting
q_degree = 3
dx = dx(metadata={'quadrature_degree': q_degree})

In [0]:
def SubMeshSave(ur, ui, file_ur, file_ui, mesh, n, stepcounter, dolfin_version):
  if dolfin_version=='1.6.0':
    V = FunctionSpace(mesh, "CG", porder)
  else:
    # For FEniCS 2016, 2017
    Ve = FiniteElement("CG", mesh.ufl_cell(), porder)
    V = FunctionSpace(mesh, Ve)
  if stepcounter % n == 0:
    ur_p = project(ur, V);
    ui_p = project(ui, V);
    ur_p.rename("Real", "label");
    ui_p.rename("Imag", "label");
    file_ur << ur_p;
    file_ui << ui_p;

def ieval(u,omega, phase):
  if omega==1:
    return u('+')*phase('+') + u('-')*phase('-');
  if omega==0:
    return u('+')*(1.-phase('+')) + u('-')*(1.-phase('-'))                              

#####################################################################################################################
#####################################################################################################################
# GEOMETRY SETTINGS
xmin, ymin, xmax, ymax = -10, -10, 10, 10

geo = "circle"

if geo == "circle":
    xc, yc = 0.0, 0.0
    R0, R1 = 5.0, 10.0
    factor =2;
    mresolution=20*factor
    segment0 = 15*factor;
    segment1 = 20*factor;

    domain0 = Circle(Point(xc, yc), R0, segment0)
    # domain  = Circle(Point(xc, yc), R1, segment1)
    domain = Rectangle(Point(xmin, ymin), Point(xmax, ymax))

    domain.set_subdomain(1, domain0)
    mesh = generate_mesh(domain, mresolution)
else:
    eps = 1e-4;
    nx, ny = 10, 10;
    factor = 1;
    mesh = RectangleMesh(Point(xmin, ymin), Point(xmax, ymax), factor*nx, factor*ny, "right")

hmin = mesh.hmin();
class Exterior(SubDomain):
    def inside (self,x,on_boundary):
        if geo=="circle":
            d = sqrt((x[0])*(x[0])+(x[1])*(x[1]));
            return d>=R0-hmin/4.0;
        else:
            return x[0]*x[0]>4*4-hmin/10. or x[1]*x[1]>4*4-hmin/10.

class Interior(SubDomain):
    def inside (self,x,on_boundary):
        exterior = Exterior()
        return not(exterior.inside(x, True))
# Mark exterior domain to 1, interior domain to 0
exterior = Exterior();
interior = Interior();
cellmarker = MeshFunction("size_t", mesh, mesh.topology().dim())
cellmarker.set_all(0)
exterior.mark(cellmarker,1)

mesh0 = SubMesh(mesh, cellmarker, 0)
mesh1 = SubMesh(mesh, cellmarker, 1)
V_DG = FunctionSpace(mesh, 'DG', 0)
dofmap_DG = V_DG.dofmap()
phase = Function(V_DG)
vol = CellVolume(mesh)
# h = 0.5*CellSize(mesh);

for cell in cells(mesh):
    phase.vector()[dofmap_DG.cell_dofs(cell.index())] = cellmarker[cell.index()];

# plot(phase); interactive(); stop
mesh_file = File("mesh.xml")
mesh_file << mesh
phase_file = File("Phi.xml")
phase_file << phase
# plot(phase); interactive(); stop;
########## END OF GEOMETRY SETTINGS
#####################################################################################################################

# parameters
#####################################################################################################################
#####################################################################################################################
porder = 1;

delta, Delta = 50000, 50000
t, T = 0, Delta+delta;
K0, K1 = 3e-3, 3e-3;
g0, g1 = 1.0, 0.0

# kappa = 5e-5; # permeability
kappa = 10.0; # permeability

nskip = 2;

bvalue = 1000;
gnorm = sqrt(bvalue)/sqrt(delta*delta*(Delta-delta/3.0));
#####################################################################################################################
#####################################################################################################################
# Sub domain for Periodic boundary condition
tol = 1e-6*mesh.hmin();
class PeriodicBoundary(SubDomain):
    # Left boundary is "target domain" G
    def inside(self, x, on_boundary):
        return bool(x[0] < xmin + tol and x[0] > xmin - tol and on_boundary)

    # Map right boundary (H) to left boundary (G)
    def map(self, x, y):
        y[0] = x[0] - (xmax-xmin)
        y[1] = x[1]

#####################################################################################################################
#####################################################################################################################
# FUNCTION SPACES
if dolfin_version=='1.6.0':
    # V = FunctionSpace(mesh , "CG", 1, constrained_domain=PeriodicBoundary());
    V = FunctionSpace(mesh , "CG", 1);
    W = MixedFunctionSpace([V, V, V, V])
    VV = MixedFunctionSpace([V, V])
else:
    # For FEniCS 2016, 2017
    Ve = FiniteElement("CG", mesh.ufl_cell(), 1)
    TH = MixedElement([Ve,Ve,Ve,Ve])
    VVe = MixedElement([Ve,Ve])
    V = FunctionSpace(mesh,Ve);
    VV = FunctionSpace(mesh,VVe);
    W = FunctionSpace(mesh, TH, constrained_domain=PeriodicBoundary())

v = TestFunction(W)
v0r, v0i, v1r, v1i = v[0], v[1], v[2], v[3]

w = TrialFunction(W);
u0r, u0i, u1r, u1i = w[0], w[1], w[2], w[3]
#####################################################################################################################
#####################################################################################################################
# Initial conditions
# one = Function(V);
Dirac_Delta = Expression("x[0]*x[0]+x[1]*x[1]<eps",eps=1e6, domain=mesh, degree=3);
Dirac_Delta = interpolate(Dirac_Delta, V);
# plot(Dirac_Delta); interactive(); stop;
# one.vector()[:] = 1;
u_0 = Function(W);
assign(u_0.sub(0), Dirac_Delta)
assign(u_0.sub(2), Dirac_Delta)
u0r_0, u0i_0, u1r_0, u1i_0 = u_0[0], u_0[1], u_0[2], u_0[3]
#####################################################################################################################
#####################################################################################################################

g = Expression(("g0","g1"), g0=g0, g1=g1,domain=mesh,degree=3);

#####################################################################################################################
#####################################################################################################################
## output files 
file_u0r = File("results/u0r.pvd")
file_u0i = File("results/u0i.pvd")
file_u1r = File("results/u1r.pvd")
file_u1i = File("results/u1i.pvd")
#####################################################################################################################
#####################################################################################################################
def FT(t, delta, Delta):
    ft1 = 1.0*(t>=0 and t<delta) 
    ft2 = -1.0*(t>=Delta and t<=Delta+delta);
    return ft1 + ft2;  

def iFT(t, delta, Delta): # integrate ft
    ft1 = t*(t>=0 and t<delta) 
    ft2 = delta*(t>=delta and t<Delta) 
    ft3 = (delta - t + Delta)*(t>=Delta and t<=Delta+delta) 
    return ft1 + ft2 + ft3;  

stepcounter = 0;
####################################################################################################################
####################################################################################################################
def FuncF(ift, gnorm, g, ur, ui, vr, vi, K):
    Fr =   ift*gnorm*(inner(g,K*grad(ui))+inner(grad(ui),K*g))*vr - inner(g,K*g)*gnorm*gnorm*ift*ift*ur*vr-K*inner(grad(ur), grad(vr))
    Fi = - ift*gnorm*(inner(g,K*grad(ur))+inner(grad(ur),K*g))*vi - inner(g,K*g)*gnorm*gnorm*ift*ift*ui*vi-K*inner(grad(ui), grad(vi))
    return Fr + Fi

def inner_interface(ift, gnorm, kappa, u0rm, u1rm, v0r, v1r, u0im, u1im, v0i, v1i, n, g, K0, K1):
    F_bcr  = (kappa*(u0rm-u1rm)+0.5*gnorm*ift*(u0im*inner(K0*g,n)-u1im*inner(K1*g,n)))*(v0r-v1r)
    F_bcr += -gnorm*ift*( u0im*inner(K0*g,n)+ u1im*inner(K1*g,n) )*0.5*(v0r+v1r)

    F_bci  = (kappa*(u0im-u1im)-0.5*gnorm*ift*(u0rm*inner(K0*g,n)-u1rm*inner(K0*g,n)))*(v0i-v1i)
    F_bci += gnorm*ift*(  u0rm*inner(K0*g,n)+ u1rm*inner(K1*g,n) )*0.5*(v0i+v1i)

    return F_bcr + F_bci

def outer_interface(K, n, ur, ui, vr, vi, g):
    F_bcr =  inner(K*g, n)*ui*vr 
    F_bci = -inner(K*g, n)*ur*vi
    return F_bcr + F_bci
#############################################################################################################################
#############################################################################################################################
def ThetaMethod_L(ift, gnorm, g, u0r, u0i, v0r, v0i, u1r, u1i, v1r, v1i, u0r_0, u0i_0, u1r_0, u1i_0,k, kappa, K0, K1, theta, phase,n):
    L0 = (u0r_0/k*v0r + u0i_0/k*v0i +theta*FuncF(ift, gnorm, g, u0r_0, u0i_0, v0r, v0i, K0))*(1-phase)*dx
    L1 = (u1r_0/k*v1r + u1i_0/k*v1i +theta*FuncF(ift, gnorm, g, u1r_0, u1i_0, v1r, v1i, K1))*phase*dx
    L_inner_bc  = -avg(theta*inner_interface(ift, gnorm, kappa, u0r_0, u1r_0, v0r, v1r, u0i_0, u1i_0, v0i, v1i, n, g, K0, K1))*abs(jump(phase))*dS;
    L0_outer_bc = -theta*(ift*gnorm+1e-16)*outer_interface(K0, n, u0r_0, u0i_0, v0r, v0i, g)*ds
    L1_outer_bc = -theta*(ift*gnorm+1e-16)*outer_interface(K1, n, u1r_0, u1i_0, v1r, v1i, g)*ds
    return L0+L1+L_inner_bc + L0_outer_bc+L1_outer_bc

def MassMatrices(u0r, u0i, v0r, v0i, u1r, u1i, v1r, v1i, K0, K1, kappa, theta, phase):
    m0 = (u0r*v0r   + u0i*v0i)*(1-phase)*dx
    m1 = (u1r*v1r   + u1i*v1i)*phase*dx
    M = assemble(m0+m1);
    M.ident_zeros();
    
    return M
def ThetaMethod_a1(ift, gnorm, g, u0r, u0i, v0r, v0i, u1r, u1i, v1r, v1i, u0r_0, u0i_0, u1r_0, u1i_0,k,kappa, K0, K1, theta, phase,n):
    a0 = (u0r/k*v0r   + u0i/k*v0i  -theta*FuncF(ift, gnorm, g, u0r  , u0i  , v0r, v0i, K0))*(1-phase)*dx
    a1 = (u1r/k*v1r   + u1i/k*v1i  -theta*FuncF(ift, gnorm, g, u1r  , u1i  , v1r, v1i, K1))*phase*dx
    a_inner_bc  = avg(  (theta*inner_interface(ift, gnorm, kappa, u0r  , u1r  , v0r, v1r, u0i  , u1i  , v0i, v1i, n, g, K0, K1)))*abs(jump(phase))*dS;
    a0_outer_bc = theta*(ift*gnorm+1e-16)*outer_interface(K0, n, u0r, u0i, v0r, v0i, g)*ds
    a1_outer_bc = theta*(ift*gnorm+1e-16)*outer_interface(K1, n, u1r, u1i, v1r, v1i, g)*ds
    return a0+a1+a_inner_bc + a0_outer_bc + a1_outer_bc
  
def ThetaMethod_a(ift, gnorm, g, u0r, u0i, v0r, v0i, u1r, u1i, v1r, v1i, u0r_0, u0i_0, u1r_0, u1i_0,k,kappa, K0, K1, theta, phase,n):
    a0 = ( -theta*FuncF(ift, gnorm, g, u0r  , u0i  , v0r, v0i, K0))*(1-phase)*dx
    a1 = ( -theta*FuncF(ift, gnorm, g, u1r  , u1i  , v1r, v1i, K1))*phase*dx
    a_inner_bc  = avg(  (theta*inner_interface(ift, gnorm, kappa, u0r  , u1r  , v0r, v1r, u0i  , u1i  , v0i, v1i, n, g, K0, K1)))*abs(jump(phase))*dS;
    a0_outer_bc = theta*(ift*gnorm)*outer_interface(K0, n, u0r, u0i, v0r, v0i, g)*ds
    a1_outer_bc = theta*(ift*gnorm)*outer_interface(K1, n, u1r, u1i, v1r, v1i, g)*ds
    return a0+a1 + a_inner_bc + a0_outer_bc + a1_outer_bc

  
def NoTimeMatrices(u0r, u0i, v0r, v0i, u1r, u1i, v1r, v1i, K0, K1, GX, kappa, theta, phase):
    m0 = (u0r*v0r   + u0i*v0i)*(1-phase)*dx
    m1 = (u1r*v1r   + u1i*v1i)*phase*dx
    M = assemble(m0+m1);
        
    j0 = -GX*(u0i*v0r   - u0r*v0i)*(1-phase)*dx
    j1 = -GX*(u1i*v1r   - u1r*v1i)*phase*dx
    J = assemble(j0+j1);    
    s0 = ( inner(K0*grad(u0r), grad(v0r)) + inner(K0*grad(u0i), grad(v0i)) )*(1-phase)*dx
    s1 = ( inner(K1*grad(u1r), grad(v1r)) + inner(K1*grad(u1i), grad(v1i)) )*phase*dx
    S = assemble(s0+s1)

    im = avg(  icondition(kappa, u0r  , u1r  , v0r, v1r, u0i  , u1i  , v0i, v1i) )*abs(jump(phase))*dS;
    I = assemble(im)
    
    
    M.ident_zeros();
    
    return M, J, S, I
def ThetaMethod_A(ft, gnorm, theta, k, M, J, S, I):
    return 1./k*M + ft*gnorm*theta*J + theta*S + theta*I

stepcounter = 0;
theta = 0.5;
Nsteps = 10
k = T/Nsteps;  
n = FacetNormal(mesh);


# M, J, S, I = NoTimeMatrices(u0r, u0i, v0r, v0i, u1r, u1i, v1r, v1i, K0, K1, GX, kappa, theta, phase);
M = MassMatrices(u0r, u0i, v0r, v0i, u1r, u1i, v1r, v1i, K0, K1, kappa, theta, phase)
#############################################################################################################################
#############################################################################################################################
# Theta method

while t < T: # Time-stepping loop
    if stepcounter % nskip == 0:
        print ('t=%f '%t, 'gnorm: %f'%gnorm);

    ft = FT(t, delta, Delta);
    ift = iFT(t, delta, Delta);

    L = ThetaMethod_L(ift, gnorm, g, u0r, u0i, v0r, v0i, u1r, u1i, v1r, v1i, u0r_0, u0i_0, u1r_0, u1i_0,k, kappa, K0, K1, theta, phase,n);
    a = ThetaMethod_a(ift, gnorm, g, u0r, u0i, v0r, v0i, u1r, u1i, v1r, v1i, u0r_0, u0i_0, u1r_0, u1i_0,k, kappa, K0, K1, theta, phase,n);
    # a = a + 1e-10*(u0r*v0r+u0i*v0i)*theta*dx + 1e-10*(u1r*v1r+u1i*v1i)*(1-theta)*dx
    # A = assemble(a);
    A = 1./k*M + assemble(a);
    # A = ThetaMethod_A(ft, gnorm, theta, k, M, J, S, I);

    # A.ident_zeros()
    b = assemble(L);
 
    u = Function(W)
    # solve(A,u.vector(),b, "gmres", "ilu");
    solve(A,u.vector(),b, "bicgstab");
    
    u0r_0, u0i_0, u1r_0, u1i_0 = split(u)
    
    # SubMeshSave(u0r_0, u0i_0, file_u0r, file_u0i, mesh0, nskip, stepcounter, dolfin_version);
    # SubMeshSave(u1r_0, u1i_0, file_u1r, file_u1i, mesh1, nskip, stepcounter, dolfin_version);
    
    t += k;
    stepcounter += 1;
plot(u1r_0); # plot(u0r_0); 
signal = assemble((phase*u1r_0+(1-phase)*u0r_0)*dx)/assemble(Dirac_Delta*dx);
print ('b:',bvalue, 'Signal: %.3e'%signal)
